In [1]:
import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging
from preprocessing import inception_preprocessing
from nets.inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
from nets import inception
import os
import time
from tensorflow.contrib import slim

from datasets import dataset_utils

In [2]:
#================ DATASET INFORMATION ======================
#State dataset directory where the tfrecord files are located
dataset_dir = 'drivers_data'

#State where your log file is at. If it doesn't exist, create it.
log_dir = 'log'

if not tf.gfile.Exists(log_dir):
    tf.gfile.MakeDirs(log_dir)

#State where your checkpoint file is
url = "http://download.tensorflow.org/models/inception_resnet_v2_2016_08_30.tar.gz"
checkpoints_dir = 'checkpoints'
checkpoint_file = 'checkpoints/inception_resnet_v2_2016_08_30.ckpt'

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

if not tf.gfile.Exists(checkpoint_file):
    dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

#State the image size you're resizing your images to. We will use the default inception size of 299.
image_size = inception.inception_resnet_v2.default_image_size

#State the number of classes to predict:
num_classes = 10

#State the labels file and read it
labels_file = 'drivers_data/labels.txt'
labels = open(labels_file, 'r')

#Create a dictionary to refer each label to their string name
labels_to_name = {}
for line in labels:
    label, string_name = line.split(':')
    string_name = string_name[:-1] #Remove newline
    labels_to_name[int(label)] = string_name

#Create the file pattern of your TFRecord files so that it could be recognized later on
file_pattern = 'drivers_%s_*.tfrecord'

#Create a dictionary that will help people understand your dataset better. This is required by the Dataset class later.
items_to_descriptions = {
    'image': 'A 3-channel RGB coloured driver image.',
    'label': 'A label of status of driver -- c0, c1, c2, c3, c4, c5, c6, c7, c8, c9'
}

In [3]:
#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 70

#State your batch size
batch_size = 32

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.001
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2